In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [3]:
# import the libraries as shown below
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [4]:
#Set all the Constants
BATCH_SIZE = 32
IMAGE_SIZE = [224, 224]
CHANNELS=3
EPOCHS=50

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "training",
    seed=123,
    shuffle=True,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 37898 files belonging to 3 classes.


In [5]:
class_names = dataset.class_names
class_names



['Healthy', 'Miner Disease', 'Rust Disease']

In [6]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [7]:
# CHECK DATASET DISTRIBUTION

y = np.concatenate([y for x, y in dataset], axis=0)
print(np.bincount(y))

[15192 15174  7532]


In [8]:
# SPLIT THE DATASET WITH THE FOLLOWING RATIO 7:2:1

def get_partition(ds,train_split = 0.7, validate = 0.2, shuffle = True, shuffle_size = 10000):
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed = 10)
    train_size = int(train_split*ds_size)
    val_size = int(validate*ds_size)
    train = ds.take(train_size)
    val = ds.skip(train_size).take(val_size)
    test = ds.skip(train_size).skip(val_size)
    return train,val,test  

In [9]:
train,val,test = get_partition(dataset)

In [10]:
len(train)


829

In [11]:
# DATA AUGMENTATION [THIS IS TO MAKE OUR DATASET LARGER BY FLIPPING APPLYING HORIZONTAL FLIP, ROTATION, ZOOM, AND CONSTRASTING TO THE IMAGE
# AND USING THE AUGMENTED IMAGE FOR TESTING]

data_augment = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)   
])   


In [12]:
# USING THE AUGMENTED IMAGE FOR THE MODEL TRAINING 

train = train.map(lambda x, y: (data_augment(x, training=True), y)).prefetch(buffer_size=tf.data.AUTOTUNE)

In [13]:
# PRE-PROCESS RESCALING OF THE INPUTS

preprocess_input = tf.keras.applications.resnet50.preprocess_input



In [14]:
# INITIALIZING THE PRE-TRAINED MOBILENETV2 MODEL 

IMG_SHAPE =  (224,224)+ (3,)
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
# CONVERT THE 224x224x3 IMAGE INTO A BLOCK OF FEATURES (224 is the recommended shape of resnet50)

image_batch, label_batch = next(iter(train))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [1]:
base_model.trainable = False

NameError: name 'base_model' is not defined

In [1]:
base_model.summary()

NameError: name 'base_model' is not defined

In [16]:
# ADD THE GLOBALAVERAGEPOOLING2D TO CONVERT THE FEATURES TO A SINGLE 1280-ELEMENT VECTOR PER IMAGE 

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

NameError: name 'feature_batch' is not defined

In [17]:
# ADD THE SOFTMAX FUNCTION TO GENERATE THE PREDICTIONS

prediction_layer = tf.keras.layers.Dense(10, activation="softmax")

In [18]:
# BUILD THE MODEL

inputs = tf.keras.Input(shape=(244, 244, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [19]:
# MODEL COMPILING 

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# SET THE INITIAL EPOCHS TO 30 

initial_epochs = 30

loss0, accuracy0 = model.evaluate(val)

In [ ]:
# CHECK THE INITIAL LOSS AND THE INITIAL ACCURACY OF THE MODEL

print("Initial loss: {:.2f}".format(loss0))
print("Initial accuracy: {:.2f}".format(accuracy0))

In [ ]:

# TRAIN THE MODEL [FIRST PHASE TRAINING]

history = model.fit(train,epochs=initial_epochs,validation_data=val, batch_size = 32, verbose =1)

In [ ]:
# UN-FREEZE THE TOP LAYER OF THE BASE MODEL

base_model.trainable = True

In [ ]:
# CHECK THE LAYERS IN THE MODEL

print("LAYERS IN THE BASE MODEL: ", len(base_model.layers))


# FINE TUNING

fine_tune_at = 100


# FREEZE THE LAYERS BEFORE THE MODEL FINE TUNING

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
# COMPILE THE FINE-TUNED MODEL

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)



In [ ]:
# FINAL MODEL TRAINING [SECOND PHASE TRAINING]

fine_tune_epochs = 1
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train,epochs=total_epochs, initial_epoch=history.epoch[-1], validation_data=val,batch_size = 32,verbose =1)

In [ ]:
# FINAL MODEL EVALUATION

scores = model.evaluate(test)

print('Test Loss:',scores[0] * 100,'%') 
print('Test Accuracy:',scores[1] * 100,'%')

In [ ]:
# MOBILENETV2 ACCURACY PLOT TO CHECK THE LOSS 
plt.plot(history.history['loss'], '#03080c')
plt.plot(history.history['val_loss'], '#336ea0')
plt.title('RESNET50 Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'],loc='upper right')
plt.show()

In [ ]:
# RESNET50 ACCURACY PLOT TO CHECK THE ACCURACY

plt.plot(history.history['accuracy'], '#03080c')
plt.plot(history.history['val_accuracy'], '#336ea0')
plt.title('RESNET50 Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'],loc='lower right')
plt.show()

In [ ]:
# TEST RUN THE MODEL (IT GETS RANDOM IMAGES FROM THE DATASET TEST AND PASS IT THROUGH THE MODEL)

for image_batch,label_batch in test.take(1):
    first_image = image_batch[0].numpy().astype('uint8')
    first_label = label_batch[0].numpy()
    batch_prediction = model.predict(image_batch)
    
    plt.imshow(first_image)
    print("Actual: ",class_names[first_label])
    
    print("Prediction: ",class_names[np.argmax(batch_prediction[0])])

In [ ]:
#Now run inference on few sample images

plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

In [ ]:
#Saving the Model
#We append the model to the list of models as a new version

import os
directory = "C:/Users/MARK3/Downloads/Omdena Ethiopia Coffee Diseases"
if not os.path.exists(directory):
os.makedirs(directory)
model.save(directory + "/model.h5")
model.save("C:/Users/MARK3/Downloads/Omdena Ethiopia Coffee Diseases/model.h5")